In [5]:
import cv2
import numpy as np

In [6]:
# Yolo 로드
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg") # weights, cfg파일을 불러와서 yolo의 네트워크와 연결한다. 
classes = [] #class 배열 만들기
with open("coco.names", "r") as f:                      # coco 파일을 읽어온다. 
    classes = [line.strip() for line in f.readlines()]  # 읽어온 coco 파일을 whitespace(공백라인)를 제거하여 classes 배열 안에 넣는다.

layer_names = net.getLayerNames()                       # 네트워크의 모든 레이어 이름을 가져와서 layer_names에 넣는다.
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()] # 레이어 중 출력 레이어의 인덱스를 가져와서 output_layers에 넣는다.
colors = np.random.uniform(0, 255, size=(len(classes), 3)) # 클래스의 갯수만큼 랜덤으로 BRG 배열을 생성한다. 한 사물 당 하나의 color만 사용할 수 있도록 해서 구분해야한다.

In [8]:
# 이미지 가져오기
img = cv2.imread("IVE.jpg") # opencv를 통해 이미지를 가져온다.
if img is None:
    print('Image not found or unable to open.')
else:
    img = cv2.resize(img, None, fx=1.4, fy=1.4)
# img = cv2.resize(img, None, fx=0.4, fy=0.4) # 이미지 크기를 재설정한다. 
height, width, channels = img.shape # 이미지의 속성들을 넣는다.

In [9]:
# 물체 감지
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False) # 이미지를 blob 객체로 처리한다.
net.setInput(blob) # blob 객체에 setInput 함수를 적용한다. 
outs = net.forward(output_layers) #output_layers를 네트워크 순방향으로 실행(추론)한다.

In [10]:
#정보를 화면에 표시
class_ids = []  # 인식한 사물 클래스 아이디를 넣는 배열
confidences = [] # 0에서 1까지 사물 인식에 대한 신뢰도를 넣는 배열
boxes = [] # 사물을 인식해서 그릴 상자에 대한 배열 
for out in outs:
    for detection in out:
        scores = detection[5:] #
        class_id = np.argmax(scores) # scores 중에서 최대값을 색인하여 class_id에 넣는다.
        confidence = scores[class_id] # scores 중에서 class_id에 해당하는 값을 confidence에 넣는다.
        if confidence > 0.5:  # 만약 정확도가 0.5가 넘는다면 사물이 인식되었다고 판단
            # 객체 탐지(Object detected)
            center_x = int(detection[0] * width) # 
            center_y = int(detection[1] * height) # 
            w = int(detection[2] * width) # 
            h = int(detection[3] * height) # 
            # 좌표
            x = int(center_x - w / 2) # 
            y = int(center_y - h / 2) # 
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

In [11]:
#노이즈 제거
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

In [12]:
font = cv2.FONT_HERSHEY_PLAIN # Font종류 중 하나인 FONT_HERSHEY_PLAIN(작은 크기 산세리프 폰트)를 적용한다.
for i in range(len(boxes)): 
    if i in indexes:
        x, y, w, h = boxes[i] 
        label = str(classes[class_ids[i]]) # 클래스 아이디 지정해둔 것을 label변수에 저장
        color = colors[i] #위에서 colors배열에 색상을 넣어둔 것을 color에 저장
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2) # 사각형을 그린다. 
        cv2.putText(img, label, (x, y + 30), font, 3, color, 3) # yolo에서 학습된 사물의 명칭을 출력한다.
cv2.imshow("Image", img) # 이미지 출력
cv2.waitKey(0) # waitKey() : 0 -> 키 입력이 있을 때까지 무한 대기한다. / ms 단위로 입력하면 그 단위에 따라 대기한다.
cv2.destroyAllWindows() # 열린 모든 창을 닫는다.